# Descriptive Method
## Analyzing and visualizing the data

### Importing the data

In [1]:
import os
import pandas as pd

# Read all of the CSV label files into dataframes:
dfs = []
for dirname, _, filenames in os.walk('test_labels'):
    for filename in filenames:
        df = pd.read_csv(os.path.join(dirname, filename))
        df['id'] = int(filename[:filename.index('.')])
        dfs.append(df)
for dirname, _, filenames in os.walk('train_labels'):
    for filename in filenames:
        df = pd.read_csv(os.path.join(dirname, filename))
        df['id'] = int(filename[:filename.index('.')])
        dfs.append(df)

#### The dataset provides an additional CSV file with metadata for each piece. We can use this file to filter the dataset for solo piano pieces only.

In [2]:
metadata = pd.read_csv('musicnet_metadata.csv')
metadata = metadata
df = pd.concat(dfs).merge(metadata,on='id')
df.head()

,start_time,end_time,instrument,note,start_beat,end_beat,note_value,id,composer,composition,movement,ensemble,source,transcriber,catalog_name,seconds
0,28126,46046,72,70,11.0,0.990625,Quarter,1819,Mozart,Serenade in E-flat major,4. Menuetto,Pairs Clarinet-Horn-Bassoon,Soni Ventorum Wind Quintet,Oliver Seely,K375,177
1,46046,80350,71,63,12.0,2.000000,Half,1819,Mozart,Serenade in E-flat major,4. Menuetto,Pairs Clarinet-Horn-Bassoon,Soni Ventorum Wind Quintet,Oliver Seely,K375,177
2,46046,97758,72,67,12.0,2.990625,Dotted Half,1819,Mozart,Serenade in E-flat major,4. Menuetto,Pairs Clarinet-Horn-Bassoon,Soni Ventorum Wind Quintet,Oliver Seely,K375,177
3,62942,69598,72,75,13.0,0.390625,Dotted Sixteenth,1819,Mozart,Serenade in E-flat major,4. Menuetto,Pairs Clarinet-Horn-Bassoon,Soni Ventorum Wind Quintet,Oliver Seely,K375,177
4,80350,97758,72,70,14.0,0.990625,Quarter,1819,Mozart,Serenade in E-flat major,4. Menuetto,Pairs Clarinet-Horn-Bassoon,Soni Ventorum Wind Quintet,Oliver Seely,K375,177


#### Now the data is ready for us to use in creating our visualizations.

### 1) First visualization: Pie & bar charts showing composer distribution

#### Let's apply some aggregate functions to our data and see what we can learn about which composers are included in the dataset.

In [3]:
# Count number of MIDI files for each composer:
count_midis = metadata.groupby(metadata['composer']).count().reset_index()[['composer','id']]

# Count number of individual notes for each composer:
count_notes = df.groupby(df['composer']).count().reset_index()[['composer','note']]

# Caculate number of individual notes per MIDI file for each composer:
counts = count_midis.merge(count_notes,on='composer')
counts['note-per-midi'] = counts.note / counts.id
counts

,composer,id,note,note-per-midi
0,Bach,67,62776,936.955224
1,Beethoven,157,566159,3606.108280
2,Brahms,24,131899,5495.791667
3,Cambini,9,24820,2757.777778
4,Dvorak,8,31605,3950.625000
5,Faure,4,22349,5587.250000
6,Haydn,3,6292,2097.333333
7,Mozart,24,75930,3163.750000
8,Ravel,4,21134,5283.500000
9,Schubert,30,146576,4885.866667


#### The table contains, for each composer: the number of distinct pieces, the total number of individual notes, and the number of notes per piece.

#### Let's create a visualization using this table to better understand its contents.

In [12]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(
    rows=1,
    cols=3,
    specs=[[{'type':'pie'},{'type':'pie'},{'type':'bar'}]],
    subplot_titles=('# of MIDIs per Composer','# of Notes per Composer','Notes per MIDI')
)

fig.add_trace(go.Pie(labels=counts.composer.values,values=counts.id.values,name='# of MIDIs'),row=1,col=1)
fig.add_trace(go.Pie(labels=counts.composer.values,values=counts.note.values,name='# of Notes'),row=1,col=2)
fig.add_trace(go.Bar(x=counts.composer.values,y=counts['note-per-midi'].values,name='Notes per MIDI',showlegend=False),row=1,col=3)

fig.update_layout(height=1500, width=1500)
fig.update_layout(xaxis_title="Composer")
fig.update_layout(legend=dict(yanchor='top',y=1,xanchor='left',x=-0.1))
fig.show()

### 2) Second visualization: Stacked bar chart (with colorscale) showing note pitches & octaves by composer

#### For the next visual, let's dig into the individual notes that make up the MIDI files. It should be helpful to visualize how the 3 different composers use different notes in their respective compositions.

#### First, we define a couple of functions to convert the MIDI numbers from the data into their corresponding pitches and octaves.

In [5]:
notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] 
def convertMIDIValueToPitch(df_row):
    return notes[df_row['note'] % 12]
def convertMIDIValueToOctave(df_row):
    return df_row['note'] // 12

#### Next, we apply these functions to the data.

In [6]:
df['note_pitch'] = df.apply(convertMIDIValueToPitch, axis=1)
df['note_octave'] = df.apply(convertMIDIValueToOctave, axis=1)

#### Finally, we calculate percentages to represent the prevalence of each note, by pitch and octave, in the musical works of each composer.

In [7]:
notes = df.groupby(['composer','note_pitch','note_octave']).agg({'note':'count'})['note'] / df.groupby(['composer']).agg({'note':'count'})['note'] * 100
notes_df = pd.DataFrame(notes).reset_index().rename({'composer':'Composer','note_pitch':'Note Pitch','note_octave':'Note Octave','note':'Note %'},axis='columns')
notes_df.head()

,Composer,Note Pitch,Note Octave,Note %
0,Bach,A,3,0.965337
1,Bach,A,4,3.061680
2,Bach,A,5,3.458328
3,Bach,A,6,1.717217
4,Bach,A,7,0.001593


#### Let's plot this so we can visualize the results:

In [8]:
import plotly_express as px

fig = px.bar(notes_df, x='Note Pitch', y='Note %', facet_col='Composer', color='Note Octave')
fig.update_layout(height=1500, width=1500)
fig.show()

#### The above figure makes it clearer to see how the 3 different composers utilized different notes when writing their music.

#### For example, it appears Beethoven and Schubert wrote in higher and lower octaves than Bach did, and Beethoven seems to have favored G.

### 3) Third visualization: Stacked bar chart showing note types/durations by composer

In [9]:
note_types = df.groupby(['composer','note_value']).agg({'note':'count'})['note'] / df.groupby(['composer']).agg({'note':'count'})['note'] * 100
note_types_df = pd.DataFrame(note_types).reset_index().rename({'composer':'Composer','note_value':'Note Type','note':'Note %'},axis='columns')
note_types_df.head()

,Composer,Note Type,Note %
0,Bach,Dotted Eighth,1.183573
1,Bach,Dotted Half,0.584618
2,Bach,Dotted Quarter,0.849051
3,Bach,Dotted Sixteenth,0.108322
4,Bach,Eighth,31.623550


In [10]:
fig = px.bar(note_types_df, x='Note Type', y='Note %', color='Composer')
fig.update_layout(height=1500, width=1500)
fig.show()